## Reference classification with LSTM

This is the assignment for the DL course part 3 of using LSTM model for solving text classification.

The input is the reference papers database which I have manually label and categorize.

The papers format are in PDF files -- some preprocessing manual and semi-automatic is involved.

The amount of input is not many, I start first with a small amount of dataset.

Update 13/11: two categories at moment

There are about 80 documents for each categories with one document is about 5000 words.

For the purpose of testing, I limit first to 1000 words

Reference for this code is from the course as well as [1]

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from __future__ import print_function

import re
import xml.sax.saxutils as saxutils
import keras

from bs4 import BeautifulSoup

from gensim.models.word2vec import Word2Vec

from keras.datasets import reuters
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Dropout, Activation, LSTM, Embedding
from keras.preprocessing import sequence

import numpy as np
import _pickle as pickle
from multiprocessing import cpu_count

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer

from pandas import DataFrame
from sklearn.cross_validation import train_test_split

Using TensorFlow backend.
C:\Users\k\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\k\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [30]:
max_words = 1000
max_features = 500
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32
epochs = 20

In [4]:
# Limit each newsline to a fixed number of words
document_max_num_words = 100

In [5]:
docs1 = pickle.load(open('cnn.pkl','rb'))
docs2 = pickle.load(open('device_characterization.pkl','rb'))

In [6]:
docs = []
for d1 in docs1:
    docs.append(d1)
for d2 in docs2:
    docs.append(d2)

In [18]:
alltext = []
key_label = set()
for p1, p2 in docs:
    alltext.append(p2) # = alltext + p2
    key_label.add(p1) 
i = 0
label_map = {}
for k in key_label:
    label_map[k] = i
    i = i + 1
print('total words in corpus:',len(alltext))
print(label_map)

total words in corpus: 168
{'device characterizations': 0, 'convolutional neural network': 1}


In [21]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(alltext)
num_features = 500
w2v_model = Word2Vec(alltext, size=num_features, min_count=1, window=10)
w2v_model.init_sims(replace=True)
w2v_model.save('test.word2vec')

In [22]:
number_of_documents = len(docs)
number_of_documents

168

In [23]:
num_categories = 2
X = zeros(shape=(number_of_documents, document_max_num_words, num_features)).astype(float32)
Y = zeros(shape=(number_of_documents, num_categories)).astype(float32)
empty_word = zeros(num_features).astype(float32)

In [24]:
for idx, document in enumerate(alltext):
    for jdx, word in enumerate(document):
        if jdx == document_max_num_words:
            break
            
        else:
            if word in w2v_model:
                X[idx, jdx, :] = w2v_model[word]
            else:
                X[idx, jdx, :] = empty_word

In [26]:
for idx, key in enumerate(label_map.keys()):
    Y[idx, :] = label_map[key]

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [28]:
model = Sequential()

model.add(LSTM(int(document_max_num_words*1.5), input_shape=(document_max_num_words, num_features)))
model.add(Dropout(0.3))
model.add(Dense(num_categories))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
# Train model
model.fit(X_train, Y_train, batch_size=128, nb_epoch=5, validation_data=(X_test, Y_test))

# Evaluate model
score, acc = model.evaluate(X_test, Y_test, batch_size=128)
    
print('Score: %1.4f' % score)
print('Accuracy: %1.4f' % acc)

C:\Users\k\Anaconda3\lib\site-packages\keras\models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 117 samples, validate on 51 samples
Epoch 1/5
117/117 [==============================] - 0s - loss: 0.2694 - acc: 0.9915 - val_loss: 0.1271 - val_acc: 1.0000
Epoch 2/5
117/117 [==============================] - 0s - loss: 0.1475 - acc: 0.9915 - val_loss: 0.0403 - val_acc: 1.0000
Epoch 3/5
117/117 [==============================] - 0s - loss: 0.0682 - acc: 0.9915 - val_loss: 0.0137 - val_acc: 1.0000
Epoch 4/5
117/117 [==============================] - 0s - loss: 0.0480 - acc: 0.9915 - val_loss: 0.0064 - val_acc: 1.0000
Epoch 5/5
51/51 [==============================] - 0s
Score: 0.0038
Accuracy: 1.0000


In [44]:
# vdocs = []
# for i in range(0, len(docs)):
#     k = label_map[ docs[i][0] ]
#     v = tokenizer.texts_to_sequences(docs[i][1]) # the result is list of list
#     # we need to make it list of values
#     vv = [q[0] for q in v if q != []]
#     vdocs.append((k,vv))

In [45]:
# random.seed(1000)
# random.shuffle(vdocs)

In [46]:
# test_split = 0.2
# pad = 0

# # train size
# sz1 = int(len(vdocs)*(1-test_split))
# X_train1 = []
# y_train1 = []
# for i in range(0, sz1):
#     label = vdocs[i][0]
#     y_train1.append(label)
#     txt1 = vdocs[i][1]
#     if len(txt1)<max_words:
#         for j in range(len(txt1), max_words+1):
#             txt1.append(pad)
#     else:
#         txt1 = txt1[:max_words]
#     X_train1.append(txt1)

# X_test1 = []
# y_test1 = []
# for i in range(sz1, len(vdocs)):
#     label = vdocs[i][0]
#     y_test1.append(label)
#     txt1 = vdocs[i][1]
#     if len(txt1)<max_words:
#         for j in range(len(txt1), max_words+1):
#             txt1.append(pad)
#     else:
#         txt1 = txt1[:max_words]
#     X_test1.append(txt1)

In [47]:
# X_train = np.array(X_train1)
# y_train = np.array(y_train1)
# X_test = np.array(X_test1)
# y_test = np.array(y_test1)

In [48]:
# num_features=500
# num_categories=2

In [50]:
# print('Build model...')
# model = Sequential()
# model.add(Embedding(max_features, 128))
# model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(1, activation='sigmoid'))

Build model...


In [51]:
# # try using different optimizers and different optimizer configs
# model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])

In [32]:
# print('Train...')
# model.fit(X_train, y_train,
#           batch_size=batch_size,
#           epochs=15,
#           validation_data=(X_test, y_test)
#          )
# score, acc = model.evaluate(X_test, y_test,
#                             batch_size=batch_size)
# print('Test score:', score)
# print('Test accuracy:', acc)